In [1]:
import MetaTrader5 as mt
import pandas as pd
import plotly.express as px
import matplotlib.pylab as plt
import numpy as np
import talib
from talipp.indicators import EMA, SMA, Stoch, DPO
from joblib import dump, load
from datetime import datetime
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score, confusion_matrix, classification_report
from own_functions import *

from tsfresh import extract_features, select_features
from tsfresh.utilities.dataframe_functions import roll_time_series, make_forecasting_frame
from tsfresh.utilities.dataframe_functions import impute

mt.initialize()
login = 51708234
password ="4bM&wuVJcBTnjV"
server = "ICMarketsEU-Demo"
mt.login(login,password,server)
symbol="USDCAD"
timeframe=mt.TIMEFRAME_D1

#ohcl data
ohlc_data=pd.DataFrame(mt.copy_rates_range(symbol,timeframe,datetime(2021,11,16),datetime.now()))
ohlc_data['time'] = pd.to_datetime(ohlc_data['time'], unit='s')
df=pd.DataFrame(ohlc_data)[['time','open','high','low','close']]

#Indicators
df['WILLR_15']=talib.WILLR(df['high'],df['low'],df['open'],timeperiod=15) #Williams' %R
df['WILLR_23']=talib.WILLR(df['high'],df['low'],df['open'],timeperiod=23) #Williams' %R
df['WILLR_42']=talib.WILLR(df['high'],df['low'],df['open'],timeperiod=42) #Williams' %R
df['WILLR_145']=talib.WILLR(df['high'],df['low'],df['open'],timeperiod=145) #Williams' %R


df=df.dropna()
df=df.reset_index(drop=True)
""" csv_file_path = 'ICMT5USDCAD_2010.csv'  # Specify your desired path
df.to_csv(csv_file_path, index=False)
df """

selected_signal='WILLR_15'

df_melted = df[['time', selected_signal]].copy()
df_melted["Symbols"] = symbol

df_rolled = roll_time_series(df_melted, column_id="Symbols", column_sort="time",
                             max_timeshift=20, min_timeshift=5)

X = extract_features(df_rolled.drop("Symbols", axis=1), 
                     column_id="id", column_sort="time", column_value=selected_signal, 
                     impute_function=impute, show_warnings=False)

X = X.set_index(X.index.map(lambda x: x[1]), drop=True)
X.index.name = "time"
X=X.dropna()

""" selected_signal='WILLR_23'


df_melted = df[['time',selected_signal]]
df_melted["Symbols"] = symbol

df_rolled = roll_time_series(df_melted, column_id="Symbols", column_sort="time",
                             max_timeshift=20, min_timeshift=5)

X2 = extract_features(df_rolled.drop("Symbols", axis=1), 
                     column_id="id", column_sort="time", column_value=selected_signal, 
                     impute_function=impute, show_warnings=False)

X2 = X2.set_index(X2.index.map(lambda x: x[1]), drop=True)
X2.index.name = "time"
X2=X2.dropna() """

selected_signal='WILLR_42'


df_melted = df[['time', selected_signal]].copy()
df_melted["Symbols"] = symbol

df_rolled = roll_time_series(df_melted, column_id="Symbols", column_sort="time",
                             max_timeshift=20, min_timeshift=5)

X3 = extract_features(df_rolled.drop("Symbols", axis=1), 
                     column_id="id", column_sort="time", column_value=selected_signal, 
                     impute_function=impute, show_warnings=False)

X3 = X3.set_index(X3.index.map(lambda x: x[1]), drop=True)
X3.index.name = "time"
X3=X3.dropna()

""" selected_signal='WILLR_145'


df_melted = df[['time',selected_signal]]
df_melted["Symbols"] = symbol

df_rolled = roll_time_series(df_melted, column_id="Symbols", column_sort="time",
                             max_timeshift=20, min_timeshift=5)

X4 = extract_features(df_rolled.drop("Symbols", axis=1), 
                     column_id="id", column_sort="time", column_value=selected_signal, 
                     impute_function=impute, show_warnings=False)

X4 = X4.set_index(X4.index.map(lambda x: x[1]), drop=True)
X4.index.name = "time"
X4=X4.dropna() """

df=df.set_index('time')
df = df[df.index.isin(X.index)]

selected_features = ['WILLR_15__change_quantiles__f_agg_"mean"__isabs_True__qh_0.8__ql_0.0', 
                     'WILLR_15__partial_autocorrelation__lag_2', 
                     'WILLR_15__change_quantiles__f_agg_"var"__isabs_False__qh_0.8__ql_0.0', 
                     'WILLR_15__change_quantiles__f_agg_"mean"__isabs_True__qh_0.6__ql_0.0', 'WILLR_15__change_quantiles__f_agg_"var"__isabs_False__qh_0.6__ql_0.0', 'WILLR_15__partial_autocorrelation__lag_3', 'WILLR_15__change_quantiles__f_agg_"mean"__isabs_False__qh_1.0__ql_0.6', 'WILLR_15__change_quantiles__f_agg_"mean"__isabs_False__qh_0.2__ql_0.0', 'WILLR_15__change_quantiles__f_agg_"var"__isabs_True__qh_0.8__ql_0.0', 'WILLR_15__fft_coefficient__attr_"abs"__coeff_4', 'WILLR_15__change_quantiles__f_agg_"var"__isabs_False__qh_0.6__ql_0.2', 'WILLR_15__change_quantiles__f_agg_"mean"__isabs_True__qh_0.6__ql_0.2', 'WILLR_15__change_quantiles__f_agg_"var"__isabs_True__qh_0.6__ql_0.2', 'WILLR_15__number_peaks__n_1', 'WILLR_15__change_quantiles__f_agg_"var"__isabs_True__qh_0.6__ql_0.0', 'WILLR_15__change_quantiles__f_agg_"mean"__isabs_True__qh_0.8__ql_0.2', 'WILLR_15__linear_trend__attr_"stderr"', 'WILLR_15__partial_autocorrelation__lag_6', 'WILLR_15__change_quantiles__f_agg_"var"__isabs_False__qh_0.4__ql_0.0', 'WILLR_15__ratio_beyond_r_sigma__r_2', 'WILLR_15__agg_linear_trend__attr_"stderr"__chunk_len_5__f_agg_"var"', 'WILLR_15__large_standard_deviation__r_0.25', 'WILLR_15__change_quantiles__f_agg_"mean"__isabs_True__qh_0.4__ql_0.0', 'WILLR_15__change_quantiles__f_agg_"var"__isabs_True__qh_1.0__ql_0.6', 'WILLR_15__kurtosis', 'WILLR_15__spkt_welch_density__coeff_2', 'WILLR_15__fft_coefficient__attr_"abs"__coeff_10', 'WILLR_15__change_quantiles__f_agg_"var"__isabs_False__qh_0.8__ql_0.2', 'WILLR_15__change_quantiles__f_agg_"var"__isabs_True__qh_0.8__ql_0.2', 'WILLR_15__mean_abs_change', 'WILLR_15__change_quantiles__f_agg_"mean"__isabs_True__qh_1.0__ql_0.0', 'WILLR_15__absolute_sum_of_changes', 'WILLR_15__spkt_welch_density__coeff_5', 'WILLR_15__fft_coefficient__attr_"abs"__coeff_7', 'WILLR_15__binned_entropy__max_bins_10', 'WILLR_15__ratio_beyond_r_sigma__r_0.5', 'WILLR_15__maximum', 'WILLR_15__change_quantiles__f_agg_"var"__isabs_True__qh_0.4__ql_0.0', 'WILLR_15__agg_linear_trend__attr_"stderr"__chunk_len_10__f_agg_"min"', 'WILLR_15__partial_autocorrelation__lag_5', 'WILLR_15__change_quantiles__f_agg_"mean"__isabs_False__qh_0.8__ql_0.2', 'WILLR_15__agg_linear_trend__attr_"intercept"__chunk_len_5__f_agg_"var"', 'WILLR_42__fft_coefficient__attr_"abs"__coeff_10', 'WILLR_42__number_peaks__n_1', 'WILLR_42__partial_autocorrelation__lag_2', 'WILLR_42__lempel_ziv_complexity__bins_100', 'WILLR_42__agg_linear_trend__attr_"rvalue"__chunk_len_5__f_agg_"var"', 'WILLR_42__number_peaks__n_3', 'WILLR_42__partial_autocorrelation__lag_4', 'WILLR_42__binned_entropy__max_bins_10', 'WILLR_42__partial_autocorrelation__lag_3', 'WILLR_42__agg_linear_trend__attr_"stderr"__chunk_len_5__f_agg_"var"', 'WILLR_42__agg_linear_trend__attr_"stderr"__chunk_len_5__f_agg_"min"', 'WILLR_42__change_quantiles__f_agg_"var"__isabs_False__qh_0.6__ql_0.0', 'WILLR_42__change_quantiles__f_agg_"var"__isabs_True__qh_0.6__ql_0.0', 'WILLR_42__change_quantiles__f_agg_"var"__isabs_True__qh_0.8__ql_0.2', 'WILLR_42__spkt_welch_density__coeff_2', 'WILLR_42__change_quantiles__f_agg_"mean"__isabs_True__qh_0.8__ql_0.2', 'WILLR_42__change_quantiles__f_agg_"var"__isabs_False__qh_0.8__ql_0.2', 'WILLR_42__agg_linear_trend__attr_"stderr"__chunk_len_5__f_agg_"mean"', 'WILLR_42__change_quantiles__f_agg_"var"__isabs_False__qh_0.8__ql_0.0']

# Filter selected_features based on what's available in X
selected_features_X = [feature for feature in selected_features if feature in X.columns]

# Filter selected_features based on what's available in X3
selected_features_X3 = [feature for feature in selected_features if feature in X3.columns]

    # Then, use these filtered lists to select columns from X and X3
X_filtered = X[selected_features_X]
X3_filtered = X3[selected_features_X3]

df = df.merge(X_filtered, left_index=True, right_index=True, how='left')
df = df.merge(X3_filtered, left_index=True, right_index=True, how='left')
df = df.drop(columns=['high','low','close','open'])
# Columns you want to move to the end
columns_to_move = ['WILLR_15', 'WILLR_23', 'WILLR_42', 'WILLR_145']

# Ensure these columns are in df
columns_to_move = [col for col in columns_to_move if col in df.columns]

# Other columns (excluding the ones you're moving to the end)
other_columns = [col for col in df.columns if col not in columns_to_move]

# Reorder DataFrame columns
df = df[other_columns + columns_to_move]


sum_fp=0
sum_tp=0

#Scale Data
sc_mt=load('scalertest.joblib')
#fig1=px.line(x_train,x=0,y=[1])
#fig1.show()
#fig2=px.line(x_train,x=0,y=[1])
#fig2.show()
x_test=sc_mt.transform(df)

# Initialise RandomForestClassifier
rf_classifier_mt = load('rf_classifiertest.joblib')

# Predict
y_pred = rf_classifier_mt.predict(x_test)
df.index = pd.to_datetime(df.index, unit='s')
df.index = df.index.strftime('%Y-%m-%d %H:%M:%S')
df_pred = pd.DataFrame(index=df.index)
df_pred['prediction'] = y_pred
df_pred.to_csv('predictionmodeltesting.csv')

c:\Users\Atharv\anaconda3\envs\my_env\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\Atharv\anaconda3\envs\my_env\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\Users\Atharv\anaconda3\envs\my_env\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
c:\Users\Atharv\anaconda3\envs\my_env\lib\site-packages\tsfresh\utilities\dataframe_functions.py:520: UserWarning: Your time stamps are not uniformly sampled, which makes rolling nonsensical in some domains.
  warnings.warn(
Feature Extraction: 100%|██████████| 30/30 [00:17<00:00,  1.74it/s]
c:\Users\Atharv\anaconda3\envs\my_env\lib\site-packages\tsfresh\utilities\dataframe_functions.py:520: UserWarning: Your time stamps are not uniformly sampled, which makes rolling nonsensical in some domains.
  warnings.warn(
Feature Extraction: 100%|██████████| 30/30 [00:19<00

In [2]:
df

,WILLR_15,WILLR_23,WILLR_42,WILLR_145,"WILLR_15__change_quantiles__f_agg_""mean""__isabs_True__qh_0.8__ql_0.0",WILLR_15__partial_autocorrelation__lag_2,"WILLR_15__change_quantiles__f_agg_""var""__isabs_False__qh_0.8__ql_0.0","WILLR_15__change_quantiles__f_agg_""mean""__isabs_True__qh_0.6__ql_0.0","WILLR_15__change_quantiles__f_agg_""var""__isabs_False__qh_0.6__ql_0.0",WILLR_15__partial_autocorrelation__lag_3,...,"WILLR_42__agg_linear_trend__attr_""stderr""__chunk_len_5__f_agg_""var""","WILLR_42__agg_linear_trend__attr_""stderr""__chunk_len_5__f_agg_""min""","WILLR_42__change_quantiles__f_agg_""var""__isabs_False__qh_0.6__ql_0.0","WILLR_42__change_quantiles__f_agg_""var""__isabs_True__qh_0.6__ql_0.0","WILLR_42__change_quantiles__f_agg_""var""__isabs_True__qh_0.8__ql_0.2",WILLR_42__spkt_welch_density__coeff_2,"WILLR_42__change_quantiles__f_agg_""mean""__isabs_True__qh_0.8__ql_0.2","WILLR_42__change_quantiles__f_agg_""var""__isabs_False__qh_0.8__ql_0.2","WILLR_42__agg_linear_trend__attr_""stderr""__chunk_len_5__f_agg_""mean""","WILLR_42__change_quantiles__f_agg_""var""__isabs_False__qh_0.8__ql_0.0"
time,,,,,,,,,,,,,,,,,,,,,
2022-06-13,-31.910995,-53.470483,-48.381353,-44.353762,14.764486,-0.781669,337.271945,7.168975,61.857368,-0.015843,...,0.000000,0.000000,26.383052,4.009733,94.477897,214.798743,12.617352,159.197569,0.000000,130.614226
2022-06-14,-16.907262,-28.388313,-28.989964,-26.576643,14.764486,-0.679761,337.271945,7.168975,61.857368,-0.015843,...,0.000000,0.000000,26.383052,4.009733,0.000000,268.814098,2.897378,0.000000,0.000000,130.614226
2022-06-15,-9.861809,-9.861809,-20.799612,-19.068111,15.704220,-0.756061,290.803338,14.764486,337.271945,-0.960510,...,0.000000,0.000000,130.614226,58.552217,26.411414,313.661467,8.036581,64.586641,0.000000,117.661066
2022-06-16,-22.424623,-22.424623,-30.511492,-27.971509,15.704220,-1.219010,290.803338,14.764486,337.271945,-0.338306,...,0.000000,0.000000,130.614226,58.552217,26.411414,481.454898,8.036581,64.586641,0.000000,117.661066
2022-06-17,-22.772630,-22.772630,-20.612903,-18.902529,13.144851,-0.951190,258.086618,15.704220,290.803338,-4.923091,...,0.000000,0.000000,117.661066,49.088375,46.259087,731.377835,11.821517,88.196601,0.000000,123.264546
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-03-25,-9.165808,-9.165808,-6.969460,-41.880460,13.912641,-0.085408,324.668628,13.856739,276.222082,0.203187,...,39.823857,7.916679,179.447322,73.111377,16.140462,2106.900155,4.222434,27.033920,5.453216,190.221772
2024-03-26,-16.220391,-16.220391,-12.333594,-43.780336,14.051445,-0.193700,326.745486,12.325102,255.620732,0.246850,...,49.171131,7.580698,162.499384,73.322824,16.012744,1764.166482,4.311923,32.505816,4.482356,190.714535
2024-03-27,-17.044284,-17.044284,-12.960063,-44.002219,14.765856,-0.105647,350.883642,13.165213,283.906534,0.115712,...,46.350260,7.520098,240.572726,87.118048,17.118716,1302.590059,4.903049,37.118253,4.500304,205.240689


In [3]:


df

,"WILLR_15__change_quantiles__f_agg_""mean""__isabs_True__qh_0.8__ql_0.0",WILLR_15__partial_autocorrelation__lag_2,"WILLR_15__change_quantiles__f_agg_""var""__isabs_False__qh_0.8__ql_0.0","WILLR_15__change_quantiles__f_agg_""mean""__isabs_True__qh_0.6__ql_0.0","WILLR_15__change_quantiles__f_agg_""var""__isabs_False__qh_0.6__ql_0.0",WILLR_15__partial_autocorrelation__lag_3,"WILLR_15__change_quantiles__f_agg_""mean""__isabs_False__qh_1.0__ql_0.6","WILLR_15__change_quantiles__f_agg_""mean""__isabs_False__qh_0.2__ql_0.0","WILLR_15__change_quantiles__f_agg_""var""__isabs_True__qh_0.8__ql_0.0","WILLR_15__fft_coefficient__attr_""abs""__coeff_4",...,"WILLR_42__change_quantiles__f_agg_""var""__isabs_True__qh_0.8__ql_0.2",WILLR_42__spkt_welch_density__coeff_2,"WILLR_42__change_quantiles__f_agg_""mean""__isabs_True__qh_0.8__ql_0.2","WILLR_42__change_quantiles__f_agg_""var""__isabs_False__qh_0.8__ql_0.2","WILLR_42__agg_linear_trend__attr_""stderr""__chunk_len_5__f_agg_""mean""","WILLR_42__change_quantiles__f_agg_""var""__isabs_False__qh_0.8__ql_0.0",WILLR_15,WILLR_23,WILLR_42,WILLR_145
time,,,,,,,,,,,,,,,,,,,,,
2022-06-13,14.764486,-0.781669,337.271945,7.168975,61.857368,-0.015843,19.463155,7.999141,183.451517,62.958013,...,94.477897,214.798743,12.617352,159.197569,0.000000,130.614226,-31.910995,-53.470483,-48.381353,-44.353762
2022-06-14,14.764486,-0.679761,337.271945,7.168975,61.857368,-0.015843,17.233444,7.999141,183.451517,62.958013,...,0.000000,268.814098,2.897378,0.000000,0.000000,130.614226,-16.907262,-28.388313,-28.989964,-26.576643
2022-06-15,15.704220,-0.756061,290.803338,14.764486,337.271945,-0.960510,11.024593,7.999141,150.293612,32.252234,...,26.411414,313.661467,8.036581,64.586641,0.000000,117.661066,-9.861809,-9.861809,-20.799612,-19.068111
2022-06-16,15.704220,-1.219010,290.803338,14.764486,337.271945,-0.338306,3.162124,7.999141,150.293612,26.704422,...,26.411414,481.454898,8.036581,64.586641,0.000000,117.661066,-22.424623,-22.424623,-30.511492,-27.971509
2022-06-17,13.144851,-0.951190,258.086618,15.704220,290.803338,-4.923091,-1.955123,7.999141,157.996521,44.990354,...,46.259087,731.377835,11.821517,88.196601,0.000000,123.264546,-22.772630,-22.772630,-20.612903,-18.902529
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-03-25,13.912641,-0.085408,324.668628,13.856739,276.222082,0.203187,0.950718,6.414385,132.447216,23.855827,...,16.140462,2106.900155,4.222434,27.033920,5.453216,190.221772,-9.165808,-9.165808,-6.969460,-41.880460
2024-03-26,14.051445,-0.193700,326.745486,12.325102,255.620732,0.246850,-0.889614,6.414385,130.983200,61.913727,...,16.012744,1764.166482,4.311923,32.505816,4.482356,190.714535,-16.220391,-16.220391,-12.333594,-43.780336
2024-03-27,14.765856,-0.105647,350.883642,13.165213,283.906534,0.115712,-0.878660,6.414385,133.913466,74.869120,...,17.118716,1302.590059,4.903049,37.118253,4.500304,205.240689,-17.044284,-17.044284,-12.960063,-44.002219
